In [1]:
import numpy as np
import math

import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature
import matplotlib.colors as colors
from matplotlib.cm import get_cmap
from matplotlib import ticker
import matplotlib.gridspec as gridspec

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

from IPython.display import Image

from tqdm import tqdm
import os 

In [2]:
N_head = 5 # first 5 lines are head lines, not include data
N_column = 4

# Injection location
Nx = 36
Ny = 21
Nz = 5
N_parcel = 3780

if Nx*Ny*Nz!=N_parcel: print('ERROR: parcel number is wrong!')
    
N_inject = 9 # how many injecting times

In [3]:
# 01

file01 = open("Lifetime01.txt", 'r')
Lines = file01.readlines()

# Strips the newline character
data = []
for line in tqdm(Lines):
    a = line.split()
    data.append(float(a[0]))
    
    
i = 0
lifetime_01 = np.zeros((Nz,Ny,Nx, N_inject))
for i_inject in tqdm(range(N_inject)):
    for ix in range(Nx):
        for iy in range(Ny):
            for iz in range(Nz):
                lifetime_01[iz,iy,ix,i_inject] = data[i]
                
                i = i+1

100%|██████████| 9/9 [00:00<00:00, 412.35it/s]


In [6]:
# first test: one time injection

Life = lifetime_01[0,:,:,0].flatten()
Life_S = lifetime_S_01[0,:,:,0].flatten()
Life_N = lifetime_N_01[0,:,:,0].flatten()

if sum(Life_S)+sum(Life_N)!=sum(Life): print("ERROR!!!")

In [7]:
N = Life.shape

(756,)

In [ ]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

c = -1*Life
A = [[-3, 1], [1, 2]]
b = [756, -756, 0]

Xs_bounds = []
A = []
for i in range(N):
    Xs_bounds.append((None, None))
    A.append([1,-1,abs(Life_N[i]-Life_S[i])-0.05*Life[i]])

from scipy.optimize import linprog
res = linprog(c, A_ub=A, b_ub=b, bounds=Xs_bounds)